# RAG EVALUATION

Nous allons évaluer notre système de RAG de tourisme au Cameroun.

1. Nous allons dans un premier temps construire le système de RAG.
2. Construire un jeu de données synthétique pour l'évaluation.
3. Evaluer avec un llm-as-judge
4. Centraliser nos évaluations sur mlflow.

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Librairies utiles et variables globales

In [ ]:
#!pip install ragatouille

In [1]:
from src.documentPreparation import prepare_rag_data
from src.agenthandler import instanciate_llm_with_huggingface, initialize_embeddings_model

In [33]:
from langchain_huggingface import HuggingFaceEndpoint
from PIL import Image

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StreamlitCallbackHandler

In [2]:
# list of documents in file folder
import os

FILES = [f"files/{f}" for f in os.listdir('files') ]
MODEL_NAME = 'all-MiniLM-L6-v2'  #'sentence-transformers/all-mpnet-base-v2' # 'sentence-transformers/all-MiniLM-L6-v2'


# 2. Préparation des documents

La préparation inclut : 

1. La lecture des documents, elle se fait en utilisant la classe PyPDFLoader de langchain

2. Le découpage du texte en petit bout appelés chunks. Nous allons utiliser le RecursiveCharacterTextSplitter de langchain.

3. Le calcul des embeddings avec un modèle pré-entrainé disponible sur huggingface

4. Le stockage de ces embeddings dans une base de données vectorielle.

Le `RecursiveCharacterTextSplitter` dans LangChain divise les documents en appliquant de manière récursive une série de séparateurs pour découper le texte en morceaux plus petits. Voici une explication étape par étape de son fonctionnement :

1. **Définir les séparateurs** :
   Vous fournissez une liste de séparateurs (par exemple, `["\n\n", "\n", " ", ""]`). Le séparateur essaiera de diviser le texte au premier séparateur de la liste. S'il ne peut pas diviser le texte en morceaux de la taille souhaitée en utilisant ce séparateur, il passera au suivant, et ainsi de suite.

2. **Division récursive** :
   Le séparateur commence par le plus grand séparateur (par exemple, `"\n\n"` pour les paragraphes) et tente de diviser le texte en morceaux plus petits que la taille spécifiée (`chunk_size`). Si les morceaux résultants sont encore trop grands, il utilisera le séparateur suivant dans la liste (par exemple, `"\n"` pour les lignes) pour diviser davantage le texte.

3. **Taille des morceaux et chevauchement** :
   Vous spécifiez une `chunk_size` (taille maximale de chaque morceau) et un `chunk_overlap` (nombre de caractères qui doivent chevaucher entre les morceaux consécutifs). Le séparateur s'assure que chaque morceau est dans la taille spécifiée et inclut le chevauchement pour maintenir le contexte entre les morceaux.

4. **Morceaux finaux** :
   Le processus continue jusqu'à ce que le texte soit divisé en morceaux qui sont tous dans la taille souhaitée. Les morceaux finaux sont ensuite retournés sous forme de liste.

Cette manière de faire le chunks assure de garder le maximum de cohérence après le découpage.

In [3]:
CHUNK_SIZE = 300
CHUNK_OVERLAP = 50
MAX_NEW_TOKENS = 500
DO_SAMPLE = True
TEMPERATURE = 0.8
TOP_P = 0.9
REPETITION_PENALTY = 1.1

In [10]:
embedding_model = initialize_embeddings_model(MODEL_NAME)

In [11]:
db = prepare_rag_data(
    list_file_path = FILES, 
    chunk_size = CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    model_name = MODEL_NAME,
    embedding_model=embedding_model,
)

Reading pdf files...
Chunking the documents...
Initializing embeddings model...
Creating vectorial db...
Index not found, generating it...


In [30]:
model = instanciate_llm_with_huggingface(
        model_name = "mistralai/Mistral-7B-Instruct-v0.3",  
        max_new_tokens = MAX_NEW_TOKENS,
        do_sample = DO_SAMPLE,
        temperature  = TEMPERATURE,
        top_p = TOP_P,
        repetition_penalty = REPETITION_PENALTY
        )

In [12]:
question = "what are the main meals in Cameroon ?"

In [15]:
docs  = db.similarity_search(
        query=question, k=20
    )

In [18]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from langchain.docstore.document import Document as LangchainDocument
from typing import List, Tuple, Optional

In [20]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [28]:
def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
    rag_prompt_template: str = RAG_PROMPT_TEMPLATE,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = rag_prompt_template.format(question=question, context=context)

    # Redact an answer
    answer = llm.invoke(final_prompt)

    return answer, relevant_docs

In [34]:
llm = HuggingFaceEndpoint(
    repo_id= MODEL_NAME,
    task="text-generation",
    max_new_tokens=1024,
    do_sample=True,
    #repetition_penalty=1.1,
    temperature=0.95,
    top_p=0.95
    
)

In [35]:
answer_with_rag(
    question = question,
    llm = llm,
    knowledge_index = db,
    reranker = None,
    num_retrieved_docs = 30,
    num_docs_final = 7,
    rag_prompt_template = RAG_PROMPT_TEMPLATE
)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Max retries exceeded with url: /models/all-MiniLM-L6-v2 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))"), '(Request ID: d779b53a-16a2-4714-87d0-b98513f41209)')